## Table of Contents
### 0. Nerual Network
### 1. nn.Module - Layer & Forward Propagation
### 2. Loss Function
### 3. Back Propagation
### 4. Update weights
**+) classes & methods for CNN** (explanations of methods I've taken in this notebook)

---

## 0. Neural Network

Neural Network는 torch.nn을 이용해서 만들 수 있다. 특히, NN class를 설계할 때에, torch.nn.Module을 상속받는 경우가 많다.

Neural Network를 설계하는 과정은 크게 다음과 같다. (data preprocessing 제외)
1. **NN 정의** - layer와 forward propagation의 방식을 NN class의 method를 통해 정의한다.
2. 준비한 데이터를 model에 학습시켜, **Loss**를 계산한다
3. **Back propagation**을 통해 각 parameter의 gradient 계산
4. **Update weight**

## 1. nn.Module - Layer & Forward Propagation

Neural Network module을 정의해야 한다. 
이 때, 거의 모든 NN module은 ``torch.nn.Module``을 상속한다.(NN module의 base class)

nn.module을 *상속*하므로, ``super`` 사용 필요

### Layer
주로 ``__init__()``에서, 필요한 layer들을 정의한다. 밑의 예시는 CNN을 build하는 과정이므로 convolution layer, full-connected layer를 정의하였다(ReLU나 Max Pooling은 함수로 표현).

### forward
Forward Propagation 과정(데이터가 input으로 들어와서 최종 결과값을 뱉는 과정)을 직접 설계해야 한다. ``torch.nn.functional`` class를 많이 활용한다.

 ``torch.nn.functional``: nn의 데이터가 거치는 다양한 종류의 함수를 담은 class로, convolution, loss, pooling, normalizing, dropout 등의 다양한 function을 가지고 있다.
    https://pytorch.org/docs/stable/nn.functional.html
    
### Dimension 주의!
다양한 layer를 만들고 forward에서 여러 연산을 취하는 과정에서, data의 dimension이 잘 맞는 지 확인하는 과정은 매우 중요하다.

nn이 학습하는 ``torch.size``는 보통 4개의 element로 이루어져 있다 - torch.size([a, b, c, d])
* a: batch size
* b: # of channel. 첫 input의 # of channel은 # of color와 같은 의미를 가진다
* c, d: channel(matrix) size

아래 코드의 dimension을 따지는 연습은 notebook의 마지막 파트인 *classes & methods for CNN*에서 다룬다.

### Why no backward?
``autograd``에 의해 backward function은 이미 define됨.
따라서, 따로 구현할 필요가 없음

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # conv layer
        self.conv1= nn.Conv2d(1, 6, 3)
        self.conv2= nn.Conv2d(6, 16, 3)
        
        # fully connected layer
        self.fc1= nn.Linear(16*6*6, 120)
        self.fc2= nn.Linear(120, 84)
        self.fc3= nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x= F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x= F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # Flattening
        x= x.view(-1, self.num_flat_features(x))
        
        # Fully connecting
        # why ReLU here? + no ReLU at last layer
        x= F.relu(self.fc1(x))
        x= F.relu(self.fc2(x))
        x= self.fc3(x)
        
        return x
    
    # flattening과정에 넣을 dimension 계산 - batch dimension을 제외한 나머지 dimension의 곱 return
    def num_flat_features(self, x):
        size= x.size()[1:] # all dimensions except the batch dimension
        num_features= 1
        for s in size:
            num_features *= s
        
        return num_features
        
    
net= Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params= list(net.parameters())
print(len(params))
print()
for i in range(len(params)):
    print("params ", i+1)
    print(params[i].size())
    print('-------------')

10

params  1
torch.Size([6, 1, 3, 3])
-------------
params  2
torch.Size([6])
-------------
params  3
torch.Size([16, 6, 3, 3])
-------------
params  4
torch.Size([16])
-------------
params  5
torch.Size([120, 576])
-------------
params  6
torch.Size([120])
-------------
params  7
torch.Size([84, 120])
-------------
params  8
torch.Size([84])
-------------
params  9
torch.Size([10, 84])
-------------
params  10
torch.Size([10])
-------------


### parameters
``torch.nn.Module.parameters()`` - **learnable parameter** 반환

<div class="alert alert-info"><h4>Question</h4><p> parameter의 size는 어떻게 결정되는걸까? 대충 홀수번째 parameter는 output/input 순으로, 짝수번쨰 parameter는 output을 size로 가지는 것 같은데... 조금 더 고민 필요 </p></div>

---

## 2. Loss Function

``torch.nn``에는 다양한 loss function이 있다 (링크: https://pytorch.org/docs/stable/nn.html#loss-functions)

* ``nn.L1Loss``: Mean Absolute Error
* ``nn.MSELoss``: Mean Squared Error
* ``nn.CrossEntropyLoss``: CrossEntropy

In [10]:
input_= torch.randn(1, 1, 32, 32)
output= net(input_)
target= torch.randn(10)
target= target.view(1, -1)

criterion= nn.MSELoss()
loss= criterion(output, target)
print(loss)

tensor(1.2659, grad_fn=<MseLossBackward>)


---

## 3. Back Propagation

Loss를 계산하였으니, back propagation을 진행해야 함.

loss.grad_fn에는 loss까지 거쳐왔던 연산들이 저장되어 있고, .backward()를 사용하여 그 연산에 사용된 tensor들의 gradient를 받을 수 있다(단 requires_grad= True인 tensor에 한함)

In [11]:
print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

next_functions에 그동안의 연산(grad_fn)이 저장됨

back propagate를 진행하기 전에, exisiting gradients를 clear해야 함

In [12]:
net.zero_grad()

print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0137,  0.0088,  0.0083,  0.0062, -0.0130, -0.0178])


---

## 4. Update wieghts

``torch.optim`` package에는 다양한 optimizing algorithm이 존재함. 
1. optimizer를 NN의 parameter를 parameter로 받아 call 하고
2. .backward()를 실행하고
3. optimizer의 .step()을 이용해 optimize!

**Optimizing Algorithm**
* Adam
* SGD
* etc..

In [13]:
import torch.optim as optim

optimizer= optim.SGD(net.parameters(), lr= 0.01)

optimizer.zero_grad()
output= net(input_)
loss= criterion(output, target)
loss.backward()

optimizer.step()

---

## + Classes & Methods for CNN

**CNN**은 크게 4가지 과정으로 설계된다.
1. Convolution layer (+ReLU)
2. Pooling layer (1-2 몇번 반복)
3. Flattening
4. Fully connected layer

### 1. Convolution layer
``torch.nn.Conv2d()``
* in_channels(int): input image의 channel 수 - conv1의 경우 color의 수
* out_channels(int): convolution 후의 channel 수
* kernel_size(int/tuple): filter 크기. square matrix를 filter로 쓸 경우 int만 써도 ok.

### 2. Pooling
``torch.nn.functional.max_pool2d()``

max pooling을 하는 method.
* input(tensor): pooling시킬 data. 보통 ReLU와 Convoluiton을 취한 data를 전달해 준다.
* weight(int/tuple): pooling의 kernel로 사용할 matrix의 size. 

### 3. Flattening
``torch.view()``

numpy의 reshape method에 해당

CNN의 학습 과정에서는 Flattening에 사용한다.

(-1, (# of channel) x (# of elements in channel))

### 4. Fully connected layer
``torch.nn.Linear()``
* in_features(int): input sample의 size
* out_feattures(int): output sample의 size

첫번째 fully connected layer의 경우, matrix가 flattening 후 전돨되므로 in_feauters = (# of channels) x (# of elements in channel)

### Dimension 따지기
(a, b, c, d)
    * a: batch size
    * b: # of channel(matrix, color)
    * c,d: matrix size

* Input: (1, 1, 32, 32) - data-dependent
* after conv1: (1, 6, 30, 30) - conv1의 kernel size가 3이므로, 양 끝 row/column 1개씩이 버려진다.
* after pooling1: (1, 6, 15, 15) - pooling의 kernel size가 2이므로, 30 % 2 = 15. ReLU는 size에 영향을 주지 못함
* after conv2: (1, 16, 13, 13)
* after pooling2: (1, 16, 6, 6)
* after flattening: (-1, 16x6x6)

<div class="alert alert-info"><h4>Question</h4><p> Pooling2에서 13/2가 7이 아니라 6이 나오나? 맨 마지막 row/column은 그냥 버리는겨? </p></div>